Looking at Kentucky county-level data to explore connections between child poverty, education outcomes, and early education access.

In [29]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# load county health rankings (national). 
df = pd.read_csv("../data/analytic_data2025_v2.csv", low_memory=False)

# filter to Kentucky
ky_df = df[df["State Abbreviation"] == "KY"]

# keep only the columns needed
columns_to_keep = [
    "5-digit FIPS Code",
    "State Abbreviation",
    "Name",                          
    "Children in Poverty raw value", 
    "High School Completion raw value"
]
ky_df = ky_df[columns_to_keep]

# rename to simple names for later joins and charts
ky_df.columns = [
    "fips",
    "state",
    "county",
    "child_poverty_pct",
    "hs_completion_pct"
]

# check the data
print(ky_df.shape)
ky_df.head()


(121, 5)


,fips,state,county,child_poverty_pct,hs_completion_pct
1022,21000,KY,Kentucky,0.202,0.8853320347
1023,21001,KY,Adair County,0.278,0.8578791192
1024,21003,KY,Allen County,0.215,0.8495460441
1025,21005,KY,Anderson County,0.122,0.9169763817
1026,21007,KY,Ballard County,0.224,0.9366835994


In [ ]:
# load preschool enrollment (KY counties, 2018–2022) and preview
preschool_df = pd.read_csv("../data/Children ages 3 to 4 enrolled in school.csv")

# keep only the columns needed
print(preschool_df.shape)
preschool_df.head()
preschool_df.columns


(1089, 5)


Index(['LocationType', 'Location', 'TimeFrame', 'DataFormat', 'Data'], dtype='object')

In [14]:
grad_df = pd.read_csv("../data/Cohort graduation rate.csv")

grad_df.head()


,LocationType,Location,TimeFrame,DataFormat,Data
0,State,Kentucky,2013,Rate,86.1
1,State,Kentucky,2014,Rate,87.5
2,State,Kentucky,2015,Rate,88
3,State,Kentucky,2016,Rate,88.6
4,State,Kentucky,2017,Rate,89.8
